In [1]:
#
#  https://nbviewer.jupyter.org/github/Pyomo/PyomoGallery/blob/master/transport/transport.ipynb
#

In [2]:
import pyomo.environ as pyo

In [3]:
# set of canning plants
plants = ['seattle', 'san-diego']
# set of markets
mkts = ['new-york', 'chicago', 'topeka']

In [4]:
# capacity of plants
cap = {'seattle': 350, 'san-diego': 600}
# markdet demand
dem = {'new-york': 325, 'chicago': 300, 'topeka': 275}

In [5]:
# distance between canning plants and cities (thousands of miles)
dtab = {
    ('seattle',  'new-york') : 2.5,
    ('seattle',  'chicago')  : 1.7,
    ('seattle',  'topeka')   : 1.8,
    ('san-diego','new-york') : 2.5,
    ('san-diego','chicago')  : 1.8,
    ('san-diego','topeka')   : 1.4,
    }

In [6]:
#  Parameter c(i,j)  transport cost in thousands of dollars per case ;
#            c(i,j) = 90 * d(i,j) ;
cost = {(k[0], k[1]) : 90 * v for (k, v) in dtab.items()}
cost

{('seattle', 'new-york'): 225.0,
 ('seattle', 'chicago'): 153.0,
 ('seattle', 'topeka'): 162.0,
 ('san-diego', 'new-york'): 225.0,
 ('san-diego', 'chicago'): 162.0,
 ('san-diego', 'topeka'): 125.99999999999999}

In [7]:
model = pyo.ConcreteModel()
model.x = pyo.Var(plants, mkts, bounds = (0.0, None))

In [8]:
def supply_rule(m, i):
    expr = sum(m.x[i, j] for j in mkts) <= cap[i]
    return expr
model.supply_cons = pyo.Constraint(plants, rule = supply_rule)

In [9]:
def demand_rule(m, j):
    expr = sum(m.x[i, j] for i in plants) >= dem[j]
    return expr
model.demand_cons = pyo.Constraint(mkts, rule = demand_rule)

In [10]:
def objective_rule(m):
    expr = sum(cost[i, j] * m.x[i, j] for i in plants for j in mkts)
    return expr
model.objective = pyo.Objective(rule = objective_rule, sense = pyo.minimize)

In [11]:
opt = pyo.SolverFactory("glpk")
results = opt.solve(model)

In [12]:
model.x.display()

x : Size=6, Index=x_index
    Key                       : Lower : Value : Upper : Fixed : Stale : Domain
     ('san-diego', 'chicago') :   0.0 :   0.0 :  None : False : False :  Reals
    ('san-diego', 'new-york') :   0.0 : 325.0 :  None : False : False :  Reals
      ('san-diego', 'topeka') :   0.0 : 275.0 :  None : False : False :  Reals
       ('seattle', 'chicago') :   0.0 : 300.0 :  None : False : False :  Reals
      ('seattle', 'new-york') :   0.0 :   0.0 :  None : False : False :  Reals
        ('seattle', 'topeka') :   0.0 :   0.0 :  None : False : False :  Reals


In [13]:
model.x['seattle', 'chicago']